# Predicting Musical Genres from Personal Streaming History

### CS156- Machine Learning
### Date: October 21, 2025.
### Professor Watson

# 1. Data Explanation

This project analyzes my personal Spotify streaming history spanning from 2023 to early 2025. The dataset represents a comprehensive digital archive of my music listening behavior, obtained directly from Spotify through their GDPR-compliant data download feature available in user privacy settings.

The data was acquired by:
1. Requesting my extended streaming history from Spotify's privacy dashboard
2. Waiting approximately 30 days for Spotify to compile the complete dataset, but got the data in three days.
3. Downloading the data package, which arrived as multiple JSON files.

I focused on the audio file which had my history since 2023. The raw dataset `Streaming_History_Audio_2023-2025_*.json`, contains an array of listening event objects with the following key attributes:

- **`ts`** (timestamp): The exact date and time when the stream occurred, in ISO 8601 format
- **`ms_played`** (milliseconds played): Duration for which the track was played, recorded in milliseconds
- **`master_metadata_track_name`**: The official name of the song as it appears in Spotify's catalog
- **`master_metadata_album_artist_name`**: The primary artist credited for the track
- **`master_metadata_album_album_name`**: The album or single from which the track originates
- **`spotify_track_uri`**: A unique identifier (URI) for each track in Spotify's system, following the format `spotify:track:<id>`
- **`reason_start`** and **`reason_end`**: Metadata about how the stream was initiated and terminated
- **`shuffle`**, **`skipped`**, **`offline`**: Boolean flags indicating playback context



# 2.Data Loading

I begin this analysis by importing the libraries for the data manipulation and visualization.

In [2]:
# Data manipulation and analysis
import pandas as pd
import numpy as np

# File system operations
import os
import json
import glob

# Data visualization
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# --- Machine Learning ---
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

print("✓ All libraries imported successfully")

✓ All libraries imported successfully


### Loading and Parsing The Data

On a high level I followed these three steps to convert the raw spotify data into csv:

1. Parse each JSON file and extract the streaming records
2. Concatenate all records into a single pandas DataFrame
3. Save the combined dataset as a CSV for easier subsequent access

Once I had transformed this raw data into workable CSV file using the scripts in ML_Pipeline/src/parse.py. The result from running the code in this file was the `combined_streaming_history.csv` file. The code below just loads it from my file directory because it already exists and then displays the first 5 row of the data.

In [3]:
# Define file paths
raw_data_pattern = '../Streaming_History_Audio_*.json'
ingested_data_dir = '../Ingested_Data'
combined_csv_path = os.path.join(ingested_data_dir, 'combined_streaming_history.csv')

# Create output directory if it doesn't exist
os.makedirs(ingested_data_dir, exist_ok=True)

# Check if combined CSV already exists
if os.path.exists(combined_csv_path):
    print(f"✓ Found existing combined dataset at '{combined_csv_path}'")
    print("  Loading from CSV...")
    df_combined = pd.read_csv(combined_csv_path)
else:
    print("✗ Combined dataset not found. Creating from raw JSON files...")

    # Find all JSON files matching the pattern
    json_files = glob.glob(raw_data_pattern)

    if not json_files:
        raise FileNotFoundError(f"No JSON files found matching pattern: {raw_data_pattern}")

    print(f"  Found {len(json_files)} JSON file(s) to process")

    # Load and combine all JSON data
    all_streaming_data = []
    for json_file in json_files:
        with open(json_file, 'r', encoding='utf-8') as f:
            file_data = json.load(f)
            all_streaming_data.extend(file_data)
            print(f"  → Loaded {len(file_data):,} records from {os.path.basename(json_file)}")

    # Convert to DataFrame
    df_combined = pd.DataFrame(all_streaming_data)

    # Save combined dataset
    df_combined.to_csv(combined_csv_path, index=False)
    print(f"\n✓ Combined dataset saved to '{combined_csv_path}'")

# Display dataset information
print(f"\n{'='*70}")
print(f"COMBINED DATASET SUMMARY")
print(f"{'='*70}")
print(f"Total streaming events: {len(df_combined):,}")
print(f"Dataset shape: {df_combined.shape[0]:,} rows × {df_combined.shape[1]} columns")
print(f"\nFirst 5 records:")
df_combined.head()

✓ Found existing combined dataset at '../Ingested_Data/combined_streaming_history.csv'
  Loading from CSV...

COMBINED DATASET SUMMARY
Total streaming events: 16,053
Dataset shape: 16,053 rows × 23 columns

First 5 records:


,ts,platform,ms_played,conn_country,ip_addr,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,episode_name,...,audiobook_uri,audiobook_chapter_uri,audiobook_chapter_title,reason_start,reason_end,shuffle,skipped,offline,offline_timestamp,incognito_mode
0,2023-08-27T01:02:32Z,windows,1265604,US,136.24.106.5,NaN,NaN,NaN,NaN,This Conversation About the 'Reading Mind' Is ...,...,NaN,NaN,NaN,remote,logout,False,False,False,NaN,False
1,2023-08-27T06:39:41Z,windows,1164082,US,12.13.248.226,NaN,NaN,NaN,NaN,This Conversation About the 'Reading Mind' Is ...,...,NaN,NaN,NaN,clickrow,endplay,False,True,False,1.693102e+09,False
2,2023-09-03T05:26:16Z,windows,3810,US,136.24.106.5,NaN,NaN,NaN,NaN,This Conversation About the 'Reading Mind' Is ...,...,NaN,NaN,NaN,playbtn,logout,False,False,False,1.693719e+09,False
3,2023-09-03T12:46:29Z,windows,4592540,US,136.24.106.5,NaN,NaN,NaN,NaN,This Conversation About the 'Reading Mind' Is ...,...,NaN,NaN,NaN,appload,logout,False,False,False,1.693740e+09,False
4,2023-09-04T00:29:31Z,ios,393360,US,172.56.209.239,Another In The Fire - Live,Hillsong UNITED,People,spotify:track:5PmHmU5AaBy9ld3bdQkD96,NaN,...,NaN,NaN,NaN,playbtn,trackdone,True,False,False,1.693787e+09,False


In [4]:
# Display column names and data types
print("Column Information:")
print(df_combined.dtypes)
print(f"\nMemory usage: {df_combined.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Check for missing values
print(f"\nMissing Values:")
missing_summary = df_combined.isnull().sum()
missing_summary = missing_summary[missing_summary > 0].sort_values(ascending=False)
if len(missing_summary) > 0:
    for col, count in missing_summary.items():
        pct = (count / len(df_combined)) * 100
        print(f"  {col}: {count:,} ({pct:.2f}%)")
else:
    print("  No missing values detected")

Column Information:
ts                                    object
platform                              object
ms_played                              int64
conn_country                          object
ip_addr                               object
master_metadata_track_name            object
master_metadata_album_artist_name     object
master_metadata_album_album_name      object
spotify_track_uri                     object
episode_name                          object
episode_show_name                     object
spotify_episode_uri                   object
audiobook_title                       object
audiobook_uri                         object
audiobook_chapter_uri                 object
audiobook_chapter_title               object
reason_start                          object
reason_end                            object
shuffle                                 bool
skipped                                 bool
offline                                 bool
offline_timestamp                  

# 3. Cleaning, Pre-processing, and Feature Engineering.

Once this file had been transformed into Workable CSV file, I discovered that it contained some of the podcasts which I had listened to, so the next step was to to remove those files, which I did. This resulted in the file `cleaned_streaming_history.csv` stripped off the podcast files. In this section, I removed all the streaming events which lacked essential information like the track name or the URI from the `combined_streaming_history.csv` dataset. In this step, too, I converted the timestamp field from string to datetime format and also converted the units from milliseconds into more interpretable unit (seconds and minutes).


In [5]:
# Define path for cleaned dataset
cleaned_csv_path = os.path.join(ingested_data_dir, 'cleaned_streaming_history.csv')

# Check if cleaned dataset already exists
if os.path.exists(cleaned_csv_path):
    print(f"✓ Found existing cleaned dataset at '{cleaned_csv_path}'")
    print("  Loading from CSV...")
    df_cleaned = pd.read_csv(cleaned_csv_path)
    # Convert timestamp column back to datetime
    df_cleaned['timestamp'] = pd.to_datetime(df_cleaned['timestamp'])
else:
    print("✗ Cleaned dataset not found. Performing data cleaning...")

    # Create a copy to avoid modifying the original
    df_cleaned = df_combined.copy()

    # Step 1: Remove rows with missing essential metadata
    initial_count = len(df_cleaned)
    df_cleaned = df_cleaned.dropna(subset=['master_metadata_track_name', 'spotify_track_uri'])
    removed_count = initial_count - len(df_cleaned)
    print(f"  → Removed {removed_count:,} rows with missing track name or URI ({removed_count/initial_count*100:.2f}%)")

    # Step 2: Convert timestamp to datetime and extract temporal features
    print("  → Converting timestamps and extracting temporal features...")
    df_cleaned['timestamp'] = pd.to_datetime(df_cleaned['ts'])
    df_cleaned['date'] = df_cleaned['timestamp'].dt.date
    df_cleaned['hour'] = df_cleaned['timestamp'].dt.hour
    df_cleaned['day_of_week'] = df_cleaned['timestamp'].dt.day_name()
    df_cleaned['month'] = df_cleaned['timestamp'].dt.month
    df_cleaned['year'] = df_cleaned['timestamp'].dt.year

    # Step 3: Convert milliseconds to seconds and minutes
    print("  → Converting time units...")
    df_cleaned['seconds_played'] = df_cleaned['ms_played'] / 1000
    df_cleaned['minutes_played'] = df_cleaned['seconds_played'] / 60

    # Step 4: Create cleaner column names
    df_cleaned['artist_name'] = df_cleaned['master_metadata_album_artist_name']
    df_cleaned['track_name'] = df_cleaned['master_metadata_track_name']
    df_cleaned['album_name'] = df_cleaned['master_metadata_album_album_name']

    # Step 5: Select relevant columns in logical order
    columns_to_keep = [
        # Identifiers
        'spotify_track_uri', 'track_name', 'artist_name', 'album_name',
        # Temporal data
        'timestamp', 'date', 'year', 'month', 'day_of_week', 'hour',
        # Playback metrics
        'ms_played', 'seconds_played', 'minutes_played',
        # Context flags
        'reason_start', 'reason_end', 'shuffle', 'skipped', 'offline',
        # Original timestamp
        'ts'
    ]

    # Keep only columns that exist in the dataset
    columns_to_keep = [col for col in columns_to_keep if col in df_cleaned.columns]
    df_cleaned = df_cleaned[columns_to_keep]

    # Save cleaned dataset
    df_cleaned.to_csv(cleaned_csv_path, index=False)
    print(f"\n✓ Cleaned dataset saved to '{cleaned_csv_path}'")

# Display cleaning results
print(f"\n{'='*70}")
print(f"CLEANED DATASET SUMMARY")
print(f"{'='*70}")
print(f"Total cleaned records: {len(df_cleaned):,}")
print(f"Dataset shape: {df_cleaned.shape[0]:,} rows × {df_cleaned.shape[1]} columns")
print(f"\nFirst 5 cleaned records:")
df_cleaned.head()

✓ Found existing cleaned dataset at '../Ingested_Data/cleaned_streaming_history.csv'
  Loading from CSV...

CLEANED DATASET SUMMARY
Total cleaned records: 12,727
Dataset shape: 12,727 rows × 34 columns

First 5 cleaned records:


,ts,platform,ms_played,conn_country,ip_addr,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,episode_name,...,date,hour,day_of_week,month,year,seconds_played,minutes_played,artist_name,track_name,album_name
0,2023-09-04T00:29:31Z,ios,393360,US,172.56.209.239,Another In The Fire - Live,Hillsong UNITED,People,spotify:track:5PmHmU5AaBy9ld3bdQkD96,NaN,...,2023-09-04,0,Monday,9,2023,393.360,6.556000,Hillsong UNITED,Another In The Fire - Live,People
1,2023-09-04T00:36:52Z,ios,353546,US,172.56.209.239,Good Grace - Live,Hillsong UNITED,People,spotify:track:7nzmXUrZwSOJPNmV0mOmEn,NaN,...,2023-09-04,0,Monday,9,2023,353.546,5.892433,Hillsong UNITED,Good Grace - Live,People
2,2023-09-04T00:40:11Z,ios,197657,US,172.56.209.239,Echoes (Till We See The Other Side) - Live,Hillsong UNITED,People,spotify:track:0oHYnQXUrFoIm0xraAmdNG,NaN,...,2023-09-04,0,Monday,9,2023,197.657,3.294283,Hillsong UNITED,Echoes (Till We See The Other Side) - Live,People
3,2023-09-04T00:41:55Z,ios,55170,US,172.56.209.239,Not Today,Hillsong UNITED,Wonder,spotify:track:33Nyq9QfKCXEQtzeg22vg7,NaN,...,2023-09-04,0,Monday,9,2023,55.170,0.919500,Hillsong UNITED,Not Today,Wonder
4,2023-09-04T00:43:46Z,ios,48599,US,172.56.209.239,Glory and Majesty,Jon Reddick,"God, Turn It Around",spotify:track:5lvrYFNaUV2eib9Tas1gZK,NaN,...,2023-09-04,0,Monday,9,2023,48.599,0.809983,Jon Reddick,Glory and Majesty,"God, Turn It Around"


With this cleaned dataset, I decided to explore the data through the summary statistics and visualizations to understand patterns in my listening history.

In [6]:
# Generate descriptive statistics for numerical columns
print("Descriptive Statistics (Numerical Columns):")
print(df_cleaned[['ms_played', 'seconds_played', 'minutes_played']].describe())

# Calculate additional summary statistics
print(f"\nAdditional Metrics:")
print(f"  Total listening time: {df_cleaned['minutes_played'].sum():,.2f} minutes ({df_cleaned['minutes_played'].sum()/60:,.2f} hours)")
print(f"  Unique tracks: {df_cleaned['track_name'].nunique():,}")
print(f"  Unique artists: {df_cleaned['artist_name'].nunique():,}")
print(f"  Unique albums: {df_cleaned['album_name'].nunique():,}")
print(f"  Date range: {df_cleaned['date'].min()} to {df_cleaned['date'].max()}")
print(f"  Tracks skipped: {df_cleaned['skipped'].sum():,} ({df_cleaned['skipped'].sum()/len(df_cleaned)*100:.2f}%)")

Descriptive Statistics (Numerical Columns):
          ms_played  seconds_played  minutes_played
count  1.272700e+04    12727.000000    12727.000000
mean   2.193484e+05      219.348441        3.655807
std    1.109287e+05      110.928706        1.848812
min    3.000000e+04       30.000000        0.500000
25%    1.572000e+05      157.200000        2.620000
50%    1.960000e+05      196.000000        3.266667
75%    2.459200e+05      245.920000        4.098667
max    1.748934e+06     1748.934000       29.148900

Additional Metrics:
  Total listening time: 46,527.46 minutes (775.46 hours)
  Unique tracks: 1,829
  Unique artists: 721
  Unique albums: 1,306
  Date range: 2023-09-04 to 2025-03-15
  Tracks skipped: 1,682 (13.22%)


Just out of personal curiousity, I posed some questions and decided to explore their answers through visualizations. These questions included:

1. Which songs have I played the most? This was to get a sense of my most frequent listen. To answe that, I counted the number of times each track appeared in the dataset and visualize the top 10 results as a bar chart.

In [7]:
# Count play frequency for each track
track_play_counts = df_cleaned['track_name'].value_counts().nlargest(10)

# Create bar chart
fig_top_songs = px.bar(
    x=track_play_counts.index,
    y=track_play_counts.values,
    labels={'x': 'Track Name', 'y': 'Number of Plays'},
    title='Top 10 Most Frequently Played Songs',
    color=track_play_counts.values,
    color_continuous_scale='Blues'
)

fig_top_songs.update_layout(
    xaxis_tickangle=-45,
    height=500,
    showlegend=False
)

fig_top_songs.show()

print("\nTop 10 Songs by Play Count:")
for i, (track, count) in enumerate(track_play_counts.items(), 1):
    print(f"  {i:2d}. {track}: {count} plays")


Top 10 Songs by Play Count:
   1. Commas: 277 plays
   2. Hide & Seek - Rema Remix: 214 plays
   3. BM - London View: 209 plays
   4. Beautiful Things: 202 plays
   5. Terminator: 189 plays
   6. Sunny Ade: 163 plays
   7. Been So Good (feat. Tiffany Hudson): 147 plays
   8. Great Things: 134 plays
   9. Sprinter: 130 plays
  10. Sure Been Good (feat. Tiffany Hudson): 116 plays


Looking at the data, I can remember that the most played song, was actually from Summer of 2024, where I went through some trauma. That song actually helped me through that time period.

While play count tells us about frequency, **total listening time** provides a different perspective. So the next question which I tried to answer was:

2. Which artists have I actually spent the most time listening to? This accounts for both the number of plays and the length of songs.

For that, I grouped the data by artist, sum up the total minutes played for each, and visualize the top 10.

In [8]:
# Calculate total listening time per artist
artist_listening_time = df_cleaned.groupby('artist_name')['minutes_played'].sum().nlargest(10).sort_values()

# Create horizontal bar chart (better for long artist names)
fig_top_artists = px.bar(
    x=artist_listening_time.values,
    y=artist_listening_time.index,
    labels={'x': 'Total Minutes Played', 'y': 'Artist Name'},
    title='Top 10 Artists by Total Listening Time',
    orientation='h',
    color=artist_listening_time.values,
    color_continuous_scale='Viridis'
)

fig_top_artists.update_layout(
    height=500,
    showlegend=False
)

fig_top_artists.show()

print("\nTop 10 Artists by Total Listening Time:")
for i, (artist, minutes) in enumerate(artist_listening_time.sort_values(ascending=False).items(), 1):
    hours = minutes / 60
    print(f"  {i:2d}. {artist}: {minutes:.2f} minutes ({hours:.2f} hours)")


Top 10 Artists by Total Listening Time:
   1. Elevation Worship: 5324.04 minutes (88.73 hours)
   2. Hillsong UNITED: 3156.41 minutes (52.61 hours)
   3. Hillsong Worship: 1956.18 minutes (32.60 hours)
   4. SYML: 1679.22 minutes (27.99 hours)
   5. Dave: 1559.11 minutes (25.99 hours)
   6. Adele: 1387.37 minutes (23.12 hours)
   7. Stormzy: 1016.12 minutes (16.94 hours)
   8. Asake: 946.39 minutes (15.77 hours)
   9. Ayra Starr: 814.45 minutes (13.57 hours)
  10. King Promise: 736.87 minutes (12.28 hours)


The last question I wanted to answer was:

3. When during the day do I listen to music the most?

In [9]:
# Aggregate by hour of day
hourly_listening = df_cleaned.groupby('hour')['minutes_played'].sum().reset_index()

# Create bar chart
fig_hourly = px.bar(
    hourly_listening,
    x='hour',
    y='minutes_played',
    labels={'hour': 'Hour of Day', 'minutes_played': 'Total Minutes Played'},
    title='Listening Activity by Hour of Day',
    color='minutes_played',
    color_continuous_scale='Sunset'
)

fig_hourly.update_layout(
    xaxis=dict(tickmode='linear', dtick=1),
    height=400,
    showlegend=False
)

fig_hourly.show()

print("\nPeak Listening Hours:")
top_hours = hourly_listening.nlargest(3, 'minutes_played')
for _, row in top_hours.iterrows():
    print(f"  {int(row['hour']):02d}:00 - {int(row['hour'])+1:02d}:00: {row['minutes_played']:.2f} minutes")


Peak Listening Hours:
  00:00 - 01:00: 3953.67 minutes
  02:00 - 03:00: 3196.83 minutes
  03:00 - 04:00: 3123.68 minutes


Once I had answered these questions, the next step in the pipeline was to get the audio preview samples of the different songs in the data set so that I could extract features from them which I'd use in my supervised learning process.

# **Obtaining the audio samples**

At first my plan was to use the Spotify API to request audio_features using the song's URI already present in my new dataframe, but the endpoint which would serve the response was already deprecated, so the next thing I thought of was using the Deezer API, but I also ran into a similar issue. With Deezer API, I either got a link to an audio file that wasn't existent or I actually got it, so I had to find an alternative source. I searched through reddit, and eventually found an npm package which helped to source the preview samples. I wrote a script in javascript that was able to take the stream of information from my data frame, find an 30 seconds audio preview sample url of the song and download it into the folder audio_samples.

The script for doing the finding the url can be found in `Ingestion/spotify_preview_finder`. The code which then downloaded the mp3 audio samples with the found url can be found in `Ingestion/extract_audio_samples.py` and stored the audio samples in the directory `Audio_Samples`.

# **Extracting Audio Features**

With the audio samples, I wrote a script to extract quantitative audio characteristics from the music track such as tempo, spectral features, MFCCs. etc, using the Librosa Library.

I also took each of the songs from the dataset and used the Spotify API to get their genres. After obtaining both features and genre_labels for the found music I combined that into another dataset `audio_features_with_genres.csv` which contained only unique songs, thus providing the benefit of no data leakage, and an honest evaluation i.e. the performance of the models would reflect true generalization.


The code below loads the dataset. This dataset formed the dataset used for my model construction moving forward.

In [10]:
# Load the dataset containing unique tracks and their audio features
unique_tracks_path = '../Extracted_Features/audio_features_with_genres.csv'

print(f"Loading unique tracks dataset from: {unique_tracks_path}")
df_tracks = pd.read_csv(unique_tracks_path)

print(f"\n{'='*70}")
print(f"UNIQUE TRACKS DATASET - SUMMARY")
print(f"{'='*70}")
print(f"Total unique tracks: {len(df_tracks):,}")
print(f"Total columns (metadata + features): {len(df_tracks.columns)}")

# Display genre distribution
if 'genre' in df_tracks.columns:
    print(f"\n--- Genre Distribution ---")
    genre_counts = df_tracks['genre'].value_counts()
    print(f"Number of unique genres: {len(genre_counts)}")
    print(f"\nTop 10 genres:")
    display(genre_counts.head(10))
else:
    print("\n⚠ Warning: 'genre' column not found in dataset")

print(f"\nSample of the dataset (metadata columns):")
display_cols = ['track_name', 'artist_name', 'genre', 'duration', 'tempo']
display(df_tracks[display_cols].head())

Loading unique tracks dataset from: ../Extracted_Features/audio_features_with_genres.csv

UNIQUE TRACKS DATASET - SUMMARY
Total unique tracks: 1,853
Total columns (metadata + features): 33

--- Genre Distribution ---
Number of unique genres: 118

Top 10 genres:


genre
Unknown           422
worship           366
afrobeats         259
lo-fi             112
african gospel     67
gospel             56
uk drill           56
christian          41
soft pop           34
new age            33
Name: count, dtype: int64


Sample of the dataset (metadata columns):


,track_name,artist_name,genre,duration,tempo
0,SNAP,Rosa Linn,Unknown,29.712653,172.265625
1,Lord Send Revival - Live,Hillsong Young & Free,worship,29.712653,99.384014
2,Somewhere Only We Know,Gustixa,Unknown,22.328027,86.132812
3,Happier,Marshmello,edm,29.712653,89.102909
4,Firm Foundation (He Won't) [feat. Cody Carnes],Maverick City Music,worship,29.712653,161.499023


With this, I was ready for working on my analysis. Since, this was a classification task, Supervised learning, I decided to look at three different models which I discuss later.

To properly evaluate our machine learning models, we must split our **unique tracks** into three distinct sets:

1. **Training Set (70%)**: Used to train the model parameters. The model learns patterns and relationships from this data.

2. **Validation Set (15%)**: Used during model development to:
   - Tune hyperparameters (e.g., number of trees, learning rate)
   - Perform early stopping to prevent overfitting
   - Compare different model architectures

3. **Test Set (15%)**: A completely held-out set used **only** for final evaluation. This simulates real-world performance on unseen data.

**Critical Principles**:
- The test set must remain untouched until final evaluation to provide an unbiased estimate of model performance
- We're splitting **unique tracks**, not listening events - this ensures no track appears in multiple sets
- Each set will have different songs, ensuring the model truly learns to generalize


From the results in the code cell above, we see that there were many genres which had fewer tracks than others. Thus, because of this imbalance, I decided to use a **stratified sampling** to ensure each split contains the same proportion of each genre as the original dataset. The benefit of choosing this splitting strategy was to prevent the model from being trained on an unrepresentative sample.

To really take advantage of the stratified splitting, I filtered my dataset to genres with at least 10 tracks to ensure statistical validity.

**Why 10 Tracks Per Genre?**

Before we can split our data for training, we must address a critical statistical constraint: **minimum sample size per class**.

Our dataset exhibits what's known as a **long-tail distribution**. Looking at our genre counts:
- A few genres dominate: "Unknown" (422 tracks), "worship" (366 tracks), "afrobeats" (259 tracks)
- Many genres are rare: 103 genres have fewer than 10 tracks each
- Some genres have only 1-2 samples

This creates a fundamental problem for machine learning: **How can we reliably train, validate, and test a model on genres with insufficient data?**

Recall my planned data split:
- Training: 70%
- Validation: 15%
- Test: 15%

For **stratified splitting** (maintaining genre proportions across splits), we need each genre to have enough samples to be distributed across all three sets while maintaining statistical validity.

**Mathematical Constraint**:

For a genre with $n$ samples and a split of proportions $(p_{train}, p_{val}, p_{test})$:

$$
n_{train} = \lfloor n \cdot p_{train} \rfloor, \quad n_{val} = \lfloor n \cdot p_{val} \rfloor, \quad n_{test} = \lfloor n \cdot p_{test} \rfloor
$$

where $\lfloor \cdot \rfloor$ denotes rounding down (floor function).

**Problem Cases**:

| Genre Samples | Train (70%) | Val (15%) | Test (15%) | Issue |
|---------------|-------------|-----------|------------|-------|
| $n = 1$ | 0-1 | 0 | 0 | Cannot split at all |
| $n = 2$ | 1 | 0 | 0-1 | No validation/test data |
| $n = 5$ | 3 | 0-1 | 0-1 | Validation/test too small |
| $n = 10$ | 7 | 1-2 | 1-2 | Minimum viable split |
| $n = 20$ | 14 | 3 | 3 | Comfortable split |

With $n < 10$, we cannot guarantee that each split receives at least one sample, making stratified splitting impossible or statistically meaningless.

**Why We Can't Train on Single-Sample Classes**

**Theoretical Issues**:

1. **No Generalization**: With only 1-2 samples of a genre in training, the model can't learn generalizable patterns. It will either:
   - Memorize those specific tracks (overfitting)
   - Ignore the genre entirely (underfitting)

2. **Unreliable Validation**: With 0-1 samples in validation, we can't assess:
   - Whether the model learned meaningful patterns
   - How well it generalizes to unseen data of that genre
   - What the true error rate is

3. **Meaningless Test Performance**: With 0-1 samples in test, we get:
   - High variance in performance metrics
   - Unreliable estimates of real-world accuracy
   - Either 0% or 100% accuracy—neither is informative

**Practical Example**:

Suppose we have a genre "jazz" with only 2 tracks. After splitting:
- Training: 1 track (track A)
- Validation: 0-1 tracks (track B or nothing)
- Test: 0-1 tracks (track B or nothing)

The model trained on track A cannot possibly learn what makes jazz "jazz" from a single example. It might learn that track A specifically is jazz, but this doesn't generalize.

#### Statistical Validity and Confidence

In statistics, we need sufficient sample size to make reliable inferences. For classification the rule of thumb is that each class should have at least 10-30 samples for basic statistical validity.

With our 70/15/15 split on $n=10$ samples:
- Train: 7 samples → Model can begin to identify patterns
- Val: 1-2 samples → Minimal feedback for hyperparameter tuning
- Test: 1-2 samples → Very rough performance estimate

This is the **absolute minimum**. Ideally, we'd want $n \geq 30$ per genre, but we're working with real-world constraints.

**Confidence Intervals**:

For a test set with $n_{test}$ samples per class, the standard error of the accuracy estimate is:

$$
SE = \sqrt{\frac{p(1-p)}{n_{test}}}
$$

where $p$ is the true accuracy. With $n_{test} = 1$:

$$
SE = \sqrt{\frac{0.5(0.5)}{1}} = 0.5 \text{ (50\% uncertainty!)}
$$

With $n_{test} = 2$:

$$
SE = \sqrt{\frac{0.5(0.5)}{2}} \approx 0.35 \text{ (35\% uncertainty)}
$$

With $n_{test} = 15$ (from a genre with 100 samples):

$$
SE = \sqrt{\frac{0.5(0.5)}{15}} \approx 0.13 \text{ (13\% uncertainty)}
$$

**This illustrates why small sample sizes yield unreliable performance estimates.**

#### The Tradeoff: Coverage vs. Quality

By filtering to genres with ≥10 samples, we face a tradeoff:

**What We Lose**:
- 103 rare genres (e.g., "classical", "jazz", "salsa" with 1 track each)
- 241 tracks (13% of dataset)
- Diversity in genre representation

**What We Gain**:
- Statistically valid train/val/test splits
- Reliable performance metrics
- Honest evaluation of model generalization
- Ability to use stratified sampling
- Confidence in our results


#### Final Dataset Composition

After filtering:
- **20 genres** (down from 118)
- **1,612 tracks** (87% of original 1,853)
- **Minimum 10 tracks per genre**, maximum 422 tracks

Each genre now has sufficient data for:
- Learning patterns in training (7+ samples)
- Tuning hyperparameters in validation (1-2+ samples)
- Evaluating performance in test (1-2+ samples)

This is a **realistic, honest dataset** for machine learning, even if it means excluding rare genres we don't have enough data to properly classify.


In [12]:
import ast

def prepare_data(df_tracks):
    """
    A comprehensive function to process the raw track data into ML-ready datasets.
    """
    # ======================================================================
    # STEP 1: FILTERING GENRES
    # ======================================================================
    print("="*70)
    print("STEP 1: FILTERING GENRES")
    print("="*70)
    print("Filtering to genres with at least 10 samples...")
    genre_counts = df_tracks['genre'].value_counts()
    genres_to_keep = genre_counts[genre_counts >= 10].index
    df_filtered = df_tracks[df_tracks['genre'].isin(genres_to_keep)].copy()
    print(f"Genres before filtering: {len(genre_counts)}")
    print(f"Genres after filtering:  {len(genres_to_keep)}")
    print(f"Tracks remaining: {len(df_filtered):,}")

    # ======================================================================
    # STEP 2: ASSEMBLING FEATURE MATRIX (X)
    # ======================================================================
    print("\n" + "="*70)
    print("STEP 2: ASSEMBLING FEATURE MATRIX (X)")
    print("="*70)

    # Define feature columns
    global numerical_feature_cols, array_feature_cols, feature_names
    numerical_feature_cols = [
        'duration', 'tempo', 'spec_centroid_mean', 'spec_centroid_std',
        'spec_bandwidth_mean', 'spec_bandwidth_std', 'spec_rolloff_mean',
        'spec_rolloff_std', 'zero_crossing_mean', 'zero_crossing_std',
        'rms_mean', 'rms_std', 'beat_count', 'beat_tempo'
    ]
    array_feature_cols = {
        'mfcc_mean': 20, 'mfcc_std': 20, 'chroma_mean': 12, 'chroma_std': 12,
        'mel_spec_mean': 128, 'mel_spec_std': 128, 'spec_contrast_mean': 7, 'spec_contrast_std': 7
    }

    # Extract simple numerical features
    X_numerical = df_filtered[numerical_feature_cols].values
    print(f"Extracted {X_numerical.shape[1]} simple numerical features.")

    # Function to safely parse string representations of lists/arrays
    def parse_array(s):
        try:
            return np.array(ast.literal_eval(s))
        except (ValueError, SyntaxError):
            return np.zeros(sum(array_feature_cols.values())) # Return a zero array of expected total size on failure

    # Parse and stack array features
    all_array_features = []
    for col, dim in array_feature_cols.items():
        print(f"Parsing {col} ({dim} dimensions)...")
        # Apply the parsing function and stack the results
        feature_matrix = np.vstack(df_filtered[col].apply(parse_array).values)
        # Ensure the feature matrix has the correct number of dimensions
        if feature_matrix.shape[1] != dim:
            # Handle cases where parsing might result in incorrect shapes
            # This could involve padding or truncating, but for now, we'll flag it
            print(f"  Warning: Mismatch in expected dimension for {col}. Expected {dim}, got {feature_matrix.shape[1]}. Adjusting...")
            # A simple fix: truncate or pad with zeros
            adjusted_matrix = np.zeros((feature_matrix.shape[0], dim))
            min_dim = min(dim, feature_matrix.shape[1])
            adjusted_matrix[:, :min_dim] = feature_matrix[:, :min_dim]
            feature_matrix = adjusted_matrix
        all_array_features.append(feature_matrix)

    # Combine all features
    X = np.hstack([X_numerical] + all_array_features)
    print("\nFeature matrix 'X' assembled successfully.")
    print(f"Final feature matrix shape: {X.shape}")

    # Store feature names for later use
    feature_names = numerical_feature_cols.copy()
    for col, dim in array_feature_cols.items():
        feature_names.extend([f"{col}_{i}" for i in range(dim)])

    # ======================================================================
    # STEP 3: ENCODING TARGET LABELS (y)
    # ======================================================================
    print("\n" + "="*70)
    print("STEP 3: ENCODING TARGET LABELS (y)")
    print("="*70)
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(df_filtered['genre'])
    print(f"Encoded {len(label_encoder.classes_)} genres into numerical labels.")
    print(f"Target vector 'y' shape: {y.shape}")

    # ======================================================================
    # STEP 4: SPLITTING DATA
    # ======================================================================
    print("\n" + "="*70)
    print("STEP 4: SPLITTING DATA")
    print("="*70)
    # First split: 70% train, 30% temp (validation + test)
    train_indices, temp_indices = train_test_split(
        np.arange(len(X)), test_size=0.3, random_state=42, stratify=y
    )
    # Second split: 50% of temp -> 15% validation, 15% test
    val_indices, test_indices = train_test_split(
        temp_indices, test_size=0.5, random_state=42, stratify=y[temp_indices]
    )

    X_train, y_train = X[train_indices], y[train_indices]
    X_val, y_val = X[val_indices], y[val_indices]
    X_test, y_test = X[test_indices], y[test_indices]

    print(f"Training set:    {len(X_train)} samples × {X_train.shape[1]} features")
    print(f"Validation set:   {len(X_val)} samples × {X_val.shape[1]} features")
    print(f"Test set:         {len(X_test)} samples × {X_test.shape[1]} features")

    # ======================================================================
    # STEP 5: SCALING FEATURES
    # ======================================================================
    print("\n" + "="*70)
    print("STEP 5: SCALING FEATURES")
    print("="*70)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    X_test_scaled = scaler.transform(X_test)
    print("✓ Features for training, validation, and test sets have been scaled.")

    # Return all the artifacts needed for modeling
    return X_train, X_val, X_test, y_train, y_val, y_test, scaler, label_encoder, X_train_scaled, X_val_scaled, X_test_scaled

# --------------------------------------------------------------------------------
# EXECUTE THE DATA PREPARATION PIPELINE
# --------------------------------------------------------------------------------
# Execute the function and unpack all the returned objects into the global scope
X_train, X_val, X_test, y_train, y_val, y_test, scaler, label_encoder, X_train_scaled, X_val_scaled, X_test_scaled = prepare_data(df_tracks.copy())

print("\nData preparation complete and ready for model training!")

STEP 1: FILTERING GENRES
Filtering to genres with at least 10 samples...
Genres before filtering: 118
Genres after filtering:  20
Tracks remaining: 1,612

STEP 2: ASSEMBLING FEATURE MATRIX (X)
Extracted 14 simple numerical features.
Parsing mfcc_mean (20 dimensions)...
Parsing mfcc_std (20 dimensions)...
Parsing chroma_mean (12 dimensions)...
Parsing chroma_std (12 dimensions)...
Parsing mel_spec_mean (128 dimensions)...
Parsing mel_spec_std (128 dimensions)...
Parsing spec_contrast_mean (7 dimensions)...
Parsing spec_contrast_std (7 dimensions)...

Feature matrix 'X' assembled successfully.
Final feature matrix shape: (1612, 348)

STEP 3: ENCODING TARGET LABELS (y)
Encoded 20 genres into numerical labels.
Target vector 'y' shape: (1612,)

STEP 4: SPLITTING DATA
Training set:    1128 samples × 348 features
Validation set:   242 samples × 348 features
Test set:         242 samples × 348 features

STEP 5: SCALING FEATURES
✓ Features for training, validation, and test sets have been scale

## 4. Feature and Target Analysis and Visualization


Before training the models, it's essential to understand the structure and relationships within our features. This exploratory analysis will help to provide a detailed inventory and analysis of our 348 features to ensure they are suitable for machine learning. We will investigate:

1. **Understand class balance** - Are some genres overrepresented?
2.  **Feature Inventory**: A detailed breakdown of all simple and array-based features.
3.  **Feature Correlation**: A heatmap to identify and understand relationships between features.
4.  **Feature Scaling**: An analysis of feature ranges to demonstrate the necessity of standardization.


In [16]:
# Visualization 1: Genre Distribution
print(f"{'='*70}")
print(f"VISUALIZATION 1: GENRE DISTRIBUTION")
print(f"{'='*70}\n")

# Ensure df_filtered exists (prepare_data created a local df_filtered; recreate here if missing)
if 'df_filtered' not in globals():
    print("df_filtered not found in global scope — creating df_filtered from df_tracks (genres with >= 10 samples)...")
    genre_counts_all = df_tracks['genre'].value_counts()
    genres_to_keep = genre_counts_all[genre_counts_all >= 10].index
    df_filtered = df_tracks[df_tracks['genre'].isin(genres_to_keep)].copy()
    print(f"  → df_filtered created: {len(df_filtered)} tracks across {len(genres_to_keep)} genres")
else:
    print("Using existing df_filtered from global scope")

# Prepare data for visualization
genre_dist = df_filtered['genre'].value_counts().sort_values(ascending=True)

# Create horizontal bar chart for better label readability
fig_genre_dist = px.bar(
    x=genre_dist.values,
    y=genre_dist.index,
    orientation='h',
    labels={'x': 'Number of Tracks', 'y': 'Genre'},
    title='Distribution of Genres in Training Dataset (After Filtering)',
    color=genre_dist.values,
    color_continuous_scale='Viridis',
    text=genre_dist.values
)

fig_genre_dist.update_traces(texttemplate='%{text}', textposition='outside')
fig_genre_dist.update_layout(
    height=600,
    showlegend=False,
    xaxis_title='Number of Tracks',
    yaxis_title='Genre'
)

fig_genre_dist.show()

# Calculate imbalance metrics
max_count = genre_dist.max()
min_count = genre_dist.min()
imbalance_ratio = max_count / min_count

print(f"\nClass Balance Analysis:")
print(f"  Most common genre:  {genre_dist.index[-1]} ({max_count} tracks)")
print(f"  Least common genre: {genre_dist.index[0]} ({min_count} tracks)")
print(f"  Imbalance ratio:    {imbalance_ratio:.2f}:1")
print(f"  Mean tracks/genre:  {genre_dist.mean():.1f}")
print(f"  Median tracks/genre: {genre_dist.median():.1f}")

if imbalance_ratio > 10:
    print(f"\n⚠ Warning: Dataset is imbalanced (ratio > 10:1)")
    print(f"  The model may be biased toward majority classes.")
else:
    print(f"\n✓ Dataset has moderate imbalance (ratio < 10:1)")

VISUALIZATION 1: GENRE DISTRIBUTION

df_filtered not found in global scope — creating df_filtered from df_tracks (genres with >= 10 samples)...
  → df_filtered created: 1612 tracks across 20 genres



Class Balance Analysis:
  Most common genre:  Unknown (422 tracks)
  Least common genre: amapiano (11 tracks)
  Imbalance ratio:    38.36:1
  Mean tracks/genre:  80.6
  Median tracks/genre: 27.5

⚠ Warning: Dataset is imbalanced (ratio > 10:1)
  The model may be biased toward majority classes.


This visualization reveals the class imbalance in my dataset. "Unknown" and "worship" dominate with 422 and 366 tracks respectively, while genres like "afro adura" and "amapiano" have only 11 tracks each. This imbalance means:

- The model will see many more examples of majority classes during training
- Accuracy on rare genres may be lower due to limited training data
- We should pay attention to **per-class metrics** (precision, recall, F1) rather than just overall accuracy
- Class imbalance is common in real-world datasets and must be acknowledged

Next, I examined the correlation between the audio features. Doing this ensured that I wasn't using **redundant features** in my analysis, and also guarded against **multicollinearity concerns** for one of the models which I wanted to use in my analysis (Logistic Regression). The code cell below shows some of the features which are highly correlated.

In [17]:
# Visualization 1

# Create a detailed inventory of all 348 features
print(f"{'='*80}")
print(f"FEATURE INVENTORY (348 TOTAL FEATURES)")
print(f"{'='*80}")

# 1. Simple Numerical Features (14)
print(f"\n{'─'*80}\n1. SIMPLE NUMERICAL FEATURES ({len(numerical_feature_cols)} features)\n{'─'*80}")
feature_categories = {
    'Rhythm': ['duration', 'tempo', 'beat_count', 'beat_tempo'],
    'Timbre/Brightness': ['spec_centroid_mean', 'spec_centroid_std'],
    'Texture': ['spec_bandwidth_mean', 'spec_bandwidth_std'],
    'Frequency Shape': ['spec_rolloff_mean', 'spec_rolloff_std'],
    'Percussiveness': ['zero_crossing_mean', 'zero_crossing_std'],
    'Loudness': ['rms_mean', 'rms_std']
}
for category, features in feature_categories.items():
    print(f"  • {category}: {', '.join(features)}")

# 2. Array-Based Features (334)
print(f"\n{'─'*80}\n2. ARRAY-BASED FEATURES (334 features)\n{'─'*80}")
total_array_feats = sum(array_feature_cols.values())
print(f"These are multi-dimensional vectors capturing complex audio patterns:\n")
print(f"  • Mel-Frequency Cepstral Coefficients (MFCCs): {array_feature_cols['mfcc_mean'] + array_feature_cols['mfcc_std']} features")
print(f"    (Captures timbre and vocal characteristics)\n")
print(f"  • Chroma Features: {array_feature_cols['chroma_mean'] + array_feature_cols['chroma_std']} features")
print(f"    (Captures harmonic/melodic content - which notes are played)\n")
print(f"  • Mel Spectrogram: {array_feature_cols['mel_spec_mean'] + array_feature_cols['mel_spec_std']} features")
print(f"    (Energy across 128 frequency bins on a human-perceived scale)\n")
print(f"  • Spectral Contrast: {array_feature_cols['spec_contrast_mean'] + array_feature_cols['spec_contrast_std']} features")
print(f"    (Difference between peaks and valleys in the sound spectrum)\n")

print(f"✓ Feature inventory complete.")


# Visualization 2: Feature Correlation Matrix

# --- Feature Analysis 1: Correlation Heatmap ---
print(f"\n{'='*80}")
print("FEATURE ANALYSIS 1: CORRELATION OF NUMERICAL FEATURES")
print(f"{'='*80}")

# Calculate the correlation matrix for the simple numerical features
corr_matrix = pd.DataFrame(X_train_scaled, columns=feature_names).loc[:, numerical_feature_cols].corr()

# Create the heatmap
fig_corr = px.imshow(
    corr_matrix,
    text_auto=".2f",
    aspect="auto",
    title="Correlation Matrix of Simple Numerical Features",
    labels=dict(color="Correlation"),
    template='plotly_dark'
)
fig_corr.update_layout(title_x=0.5)
fig_corr.show()

print("\n**Interpretation**: The heatmap shows relationships between features. For example, 'tempo' and 'beat_tempo' are perfectly correlated (1.0), as expected. High correlations (e.g., between spectral features) indicate some redundancy, which tree-based models can handle well.")

FEATURE INVENTORY (348 TOTAL FEATURES)

────────────────────────────────────────────────────────────────────────────────
1. SIMPLE NUMERICAL FEATURES (14 features)
────────────────────────────────────────────────────────────────────────────────
  • Rhythm: duration, tempo, beat_count, beat_tempo
  • Timbre/Brightness: spec_centroid_mean, spec_centroid_std
  • Texture: spec_bandwidth_mean, spec_bandwidth_std
  • Frequency Shape: spec_rolloff_mean, spec_rolloff_std
  • Percussiveness: zero_crossing_mean, zero_crossing_std
  • Loudness: rms_mean, rms_std

────────────────────────────────────────────────────────────────────────────────
2. ARRAY-BASED FEATURES (334 features)
────────────────────────────────────────────────────────────────────────────────
These are multi-dimensional vectors capturing complex audio patterns:

  • Mel-Frequency Cepstral Coefficients (MFCCs): 40 features
    (Captures timbre and vocal characteristics)

  • Chroma Features: 24 features
    (Captures harmonic/mel


**Interpretation**: The heatmap shows relationships between features. For example, 'tempo' and 'beat_tempo' are perfectly correlated (1.0), as expected. High correlations (e.g., between spectral features) indicate some redundancy, which tree-based models can handle well.


The librosa library gave me these audio features out of the box, so I wanted to know if one could really guess what genre a song is just by looking at its musical features. Basically, I wanted to know if these features were discriminative. For example, if Pop, Rock, and Jazz all have the same average tempo, then tempo cannot discriminate between the genres.

How I achieved that was by making a box plot of the features. The code cell and the result is shown below

In [19]:
# Visualization 3: Feature Distributions by Genre
print(f"\n{'='*70}")
print(f"VISUALIZATION 3: FEATURE DISTRIBUTIONS BY GENRE")
print(f"{'='*70}\n")

# Ensure df_filtered exists (created earlier in the notebook). If not, create it from df_tracks.
if 'df_filtered' not in globals():
    print("df_filtered not found in global scope — creating df_filtered from df_tracks (genres with >= 10 samples)...")
    genre_counts_all = df_tracks['genre'].value_counts()
    genres_to_keep = genre_counts_all[genre_counts_all >= 10].index
    df_filtered = df_tracks[df_tracks['genre'].isin(genres_to_keep)].copy()
    print(f"  → df_filtered created: {len(df_filtered)} tracks across {len(genres_to_keep)} genres")
else:
    print("Using existing df_filtered from global scope")

# Select a few key features that are likely to differ by genre
key_features = ['tempo', 'rms_mean', 'spec_centroid_mean', 'zero_crossing_mean']

# Create subplots
fig_features = make_subplots(
    rows=2, cols=2,
    subplot_titles=[f'{feat.replace("_", " ").title()}' for feat in key_features],
    vertical_spacing=0.20,
    horizontal_spacing=0.15
)

# Use top 10 genres by count for cleaner visualization
top_genres = df_filtered['genre'].value_counts().head(10).index
df_plot = df_filtered[df_filtered['genre'].isin(top_genres)]

# Add box plots for each feature
for idx, feature in enumerate(key_features):
    row = idx // 2 + 1
    col = idx % 2 + 1

    for genre in top_genres:
        genre_data = df_plot[df_plot['genre'] == genre][feature]
        fig_features.add_trace(
            go.Box(y=genre_data, name=genre, showlegend=(idx == 0)),
            row=row, col=col
        )

fig_features.update_layout(
    height=800,
    title_text="Distribution of Key Audio Features Across Top 10 Genres",
    showlegend=True
)

fig_features.show()

# Statistical comparison: ANOVA to test if features differ significantly across genres
from scipy import stats

print(f"\nStatistical Significance Tests (ANOVA):")
print(f"Testing if feature means differ significantly across genres\n")

for feature in key_features:
    # Get data for each genre (use all genres in df_filtered)
    genre_groups = [df_filtered[df_filtered['genre'] == g][feature].values
                    for g in df_filtered['genre'].unique()]

    # Perform one-way ANOVA
    # Guard against genres with insufficient variance / single sample by filtering empty groups
    genre_groups = [g for g in genre_groups if len(g) > 1]
    if len(genre_groups) < 2:
        print(f"  {feature:25s}: Not enough groups with >1 sample to perform ANOVA")
        continue

    f_stat, p_value = stats.f_oneway(*genre_groups)

    significance = "***" if p_value < 0.001 else "**" if p_value < 0.01 else "*" if p_value < 0.05 else "n.s."

    print(f"  {feature:25s}: F = {f_stat:8.2f}, p = {p_value:.2e} {significance}")

print(f"\n  Significance codes: *** p<0.001, ** p<0.01, * p<0.05, n.s. = not significant")
print(f"  → All features with p < 0.05 are discriminative for genre classification")


VISUALIZATION 3: FEATURE DISTRIBUTIONS BY GENRE

Using existing df_filtered from global scope



Statistical Significance Tests (ANOVA):
Testing if feature means differ significantly across genres

  tempo                    : F =     5.10, p = 5.18e-12 ***
  rms_mean                 : F =    21.88, p = 3.88e-67 ***
  spec_centroid_mean       : F =    96.85, p = 5.45e-249 ***
  zero_crossing_mean       : F =    60.53, p = 2.71e-172 ***

  Significance codes: *** p<0.001, ** p<0.01, * p<0.05, n.s. = not significant
  → All features with p < 0.05 are discriminative for genre classification


The box plots reveal how different genres exhibit distinct audio characteristics:

- **Tempo**: Shows variation across genres
  - Electronic genres (EDM, lo-fi) may cluster at different tempos
  - Worship music might have slower, more consistent tempos
  - Rap/hip-hop might show distinctive rhythm patterns

- **RMS Mean (Loudness)**: Indicates average energy
  - Ambient genres (lo-fi, new age) tend to be quieter

- **Spectral Centroid (Brightness)**: Measures frequency content
  - Genres with more high-frequency content have higher centroids
  - Can distinguish "bright" vs. "dark" sounding music

- **Zero Crossing Rate (Noisiness)**: Indicates percussive/noisy content
  - Genres with more percussion show higher rates
  - Smooth, melodic genres have lower rates

**ANOVA Results**: The F-statistic and p-values tell us whether feature means differ significantly across genres. Low p-values (< 0.05) indicate that the feature is **discriminative** - it helps distinguish genres. This validates that our audio features contain genre-relevant information.


**Key findings**:
- All tested features show **highly significant differences** (p < 0.001) across genres
- This confirms our features have **discriminative power** for genre classification
- The F-statistic measures the ratio of between-group variance to within-group variance:
  - Higher F → feature varies more between genres than within genres
  - Lower F → feature varies as much within genres as between them


With more confidence in the features now, I decided to do a comparison of the feature scales to know whether or not, I'll be doing some standardizatoin. The code below shows a visualization of the feature scale and some summary statistics of the feature scale.

In [20]:
# --- Feature Analysis 2: Feature Scales ---
print(f"\n{'='*80}")
print("FEATURE ANALYSIS 2: NECESSITY OF FEATURE SCALING")
print(f"{'='*80}")

# Calculate the range (max - min) for each of the 348 features before scaling
feature_ranges = pd.DataFrame(X_train).describe().loc['max'] - pd.DataFrame(X_train).describe().loc['min']

# Create a bar plot of the feature ranges
fig_scales = px.bar(
    x=feature_ranges.index,
    y=feature_ranges.values,
    title='Range of Values for Each Feature (Before Scaling)',
    labels={'x': 'Feature Index', 'y': 'Range (Max - Min)'},
    template='plotly_dark'
)
fig_scales.update_layout(title_x=0.5)
fig_scales.show()

min_range = feature_ranges.min()
max_range = feature_ranges.max()
print(f"\n**Interpretation**: The ranges of the features vary dramatically.")
print(f"  - Smallest feature range: {min_range:.4f}")
print(f"  - Largest feature range:  {max_range:,.2f}")
print(f"  - Scale ratio: {max_range/min_range:,.0f}:1")
print("\nThis vast difference in scales makes **Standardization** (scaling to mean=0, std=1) absolutely essential for models like Logistic Regression and beneficial for all models during optimization.")


FEATURE ANALYSIS 2: NECESSITY OF FEATURE SCALING



**Interpretation**: The ranges of the features vary dramatically.
  - Smallest feature range: 0.1360
  - Largest feature range:  7,250.36
  - Scale ratio: 53,321:1

This vast difference in scales makes **Standardization** (scaling to mean=0, std=1) absolutely essential for models like Logistic Regression and beneficial for all models during optimization.


From the visualization it is quite clear that scaling would be critical for this model construction. Some features like `spec_rollof_mean` span ranges of about 8000. Others like the `zero_crossing_mean` span ranges of ~0.1 which creates a scale difference. This is important because for a Logistic regression model, Regularization penalty $\lambda \|\mathbf{w}\|^2$ penalizes all weights equally, but to fit large-scale features, weights must be small, and to fit small-scale features, weights must be large. Thus, uneven penalties lead to poor regularization.

By implementing the Standard Scaler in the pipeline for my models, the StandardScaler transforms each feature to have:
$$
x'_j = \frac{x_j - \mu_j}{\sigma_j}
$$

After standardization:
- All features have mean $\mu = 0$
- All features have standard deviation $\sigma = 1$
- Scale differences are eliminated
- Optimization and modeling become fair and efficient

# 5. Model Selection and Mathematical Underpinnings



---

## Section 5: Model Selection and Mathematical Foundations

### 5.1 Overview: Supervised Multi-Class Classification

This analysis addresses a **supervised multi-class classification** problem where the goal is to predict which of **20 genre families** a music track belongs to, based solely on its **78 audio features** extracted from 30-second preview clips.

#### 5.1.1 Problem Formulation

Given:
- **Training data**: $\mathcal{D} = \{(\mathbf{x}_i, y_i)\}_{i=1}^{N}$ where $N = 1{,}128$ training samples
- **Feature vectors**: $\mathbf{x}_i \in \mathbb{R}^{348}$ (audio features: MFCC mean/std [40], chroma mean/std [24], spectral features [8], rhythmic features [6])
- **Labels**: $y_i \in \{0, 1, ..., 19\}$ (20 genre classes)

Objective:
- Learn a function $f: \mathbb{R}^{348} \rightarrow \{0, 1, ..., 19\}$ that accurately predicts genre labels for unseen tracks

#### 5.1.2 Model Selection Strategy

I will train and compare **three different architectures**, each representing distinct approaches to supervised learning:

1. **Logistic Regression** - Linear probabilistic baseline (multinomial extension)
2. **Random Forest** - Ensemble method using bagging (bootstrap aggregation)
3. **Gradient Boosting** - Ensemble method using sequential boosting

These models progress from simple linear assumptions to complex non-linear transformations, allowing systematic comparison of model sophistication versus performance on high-dimensional audio data.

#### 5.1.3 Evaluation Strategy

All models will be evaluated using:
- **K-Fold Cross-Validation** (5-fold stratified) on training set for robust performance estimation
- **Hold-out test set** (15% of data) for final unbiased evaluation
- **Multiple metrics**: Accuracy, Precision, Recall, F1-Score (macro and weighted averages)
- **Confusion matrices**: To understand per-genre performance and misclassification patterns

---

### 5.2 Model 1: Logistic Regression (Multinomial Extension)

#### 5.2.1 Mathematical Foundation

**Logistic Regression** generalizes binary classification to handle multi-class problems using the **softmax function**. For $K = 20$ genre classes, the model learns:
- Weight matrix: $\mathbf{W} \in \mathbb{R}^{K \times d}$ where $d = 348$ features
- Bias vector: $\mathbf{b} \in \mathbb{R}^{K}$

**Probability distribution** over classes given input $\mathbf{x}$:

$$
P(y = k | \mathbf{x}; \mathbf{W}, \mathbf{b}) = \frac{\exp(\mathbf{w}_k^T \mathbf{x} + b_k)}{\sum_{j=1}^{K} \exp(\mathbf{w}_j^T \mathbf{x} + b_j)} = \text{softmax}(\mathbf{W}\mathbf{x} + \mathbf{b})_k
$$

where $\mathbf{w}_k$ is the $k$-th row of $\mathbf{W}$ (weight vector for class $k$).

**Prediction**: Choose the class with maximum probability:

$$
\hat{y} = \arg\max_{k} P(y = k | \mathbf{x})
$$

#### 5.2.2 Loss Function: Cross-Entropy

The model is trained by minimizing the **categorical cross-entropy loss**:

$$
\mathcal{L}(\mathbf{W}, \mathbf{b}) = -\frac{1}{N} \sum_{i=1}^{N} \sum_{k=1}^{K} \mathbb{1}\{y_i = k\} \log P(y = k | \mathbf{x}_i; \mathbf{W}, \mathbf{b})
$$

where $\mathbb{1}\{\cdot\}$ is the indicator function (1 if true, 0 if false).

**With L2 regularization** to prevent overfitting:

$$
\mathcal{L}_{reg}(\mathbf{W}, \mathbf{b}) = \mathcal{L}(\mathbf{W}, \mathbf{b}) + \lambda \|\mathbf{W}\|_F^2
$$

where $\|\mathbf{W}\|_F$ is the Frobenius norm and $\lambda$ controls regularization strength.

#### 5.2.3 Optimization

The model is trained using **gradient descent** or variants (e.g., L-BFGS):

$$
\mathbf{W} \leftarrow \mathbf{W} - \eta \nabla_{\mathbf{W}} \mathcal{L}_{reg}
$$

where $\eta$ is the learning rate.

**Gradient with respect to $\mathbf{w}_k$**:

$$
\nabla_{\mathbf{w}_k} \mathcal{L} = \frac{1}{N} \sum_{i=1}^{N} \left( P(y = k | \mathbf{x}_i) - \mathbb{1}\{y_i = k\} \right) \mathbf{x}_i
$$

#### 5.2.4 Why Logistic Regression?

**Advantages**:
- Fast training and prediction (linear complexity in features)
- Provides calibrated probability estimates
- Interpretable: weights indicate feature importance for each genre
- Baseline for comparison with non-linear models

**Limitations**:
- Assumes linear decision boundaries (may underfit complex patterns)
- Cannot capture feature interactions automatically
- Sensitive to class imbalance (38:1 ratio in our data)
- Requires feature scaling (the features span ~60,000:1 scale differences)

**Expected Performance**: Will serve as a fast baseline, likely achieving moderate accuracy but may struggle with genres that require non-linear feature combinations.

---

### 5.3 Model 2: Random Forest

#### 5.3.1 Decision Tree Foundation

A **decision tree** recursively partitions the feature space by selecting optimal splits:

At each node, choose feature $j$ and threshold $t$ to maximize **information gain**:

$$
\text{InfoGain}(j, t) = \text{Impurity}(S) - \sum_{v \in \{L, R\}} \frac{|S_v|}{|S|} \text{Impurity}(S_v)
$$

where $S$ is the current node's samples, $S_L$ and $S_R$ are left and right children after splitting on feature $j$ at threshold $t$.

**Gini Impurity** measures node purity:

$$
\text{Gini}(S) = 1 - \sum_{k=1}^{K} p_k^2
$$

where $p_k = \frac{1}{|S|}\sum_{i \in S} \mathbb{1}\{y_i = k\}$ is the proportion of class $k$ in node $S$.

- Gini = 0: All samples in $S$ belong to one class (pure)
- Gini = $1 - \frac{1}{K}$: Classes uniformly distributed (maximum impurity for $K$ classes)

#### 5.3.2 Random Forest Algorithm

**Random Forest** is an ensemble of $T$ decision trees trained with **bagging** (bootstrap aggregation) and **feature randomization**:

**Training** (for each tree $t = 1, ..., T$):
1. **Bootstrap sample**: Randomly sample $N$ training examples with replacement from $\mathcal{D}$
2. **Grow tree**: At each split, randomly select $m = \sqrt{d} \approx \sqrt{348} \approx 19$ features and find the best split among them
3. **No pruning**: Grow trees until leaves are pure or contain $< n_{min}$ samples

**Prediction** (majority vote):

$$
\hat{y} = \text{mode}\{h_1(\mathbf{x}), h_2(\mathbf{x}), ..., h_T(\mathbf{x})\}
$$

where $h_t(\mathbf{x})$ is the prediction of tree $t$.

#### 5.3.3 Why Randomness Reduces Overfitting

Two sources of randomness ensure **diversity** among trees:

1. **Bagging**: Each tree trained on different bootstrap sample (~63% of data, some samples repeated)
2. **Feature randomization**: Each split considers only $m \ll d$ random features

**Bias-Variance Tradeoff**:
- Individual deep trees: Low bias (flexible), high variance (overfit to training data)
- Averaging predictions: Reduces variance without increasing bias
- Decorrelated trees via randomness: Maximizes variance reduction

#### 5.3.4 Why Random Forest?

**Advantages**:
- Captures non-linear patterns and feature interactions naturally
- Robust to outliers and noise
- Handles high-dimensional data ($d = 78$) without explicit feature selection
- Provides feature importance measures (mean decrease in Gini impurity)
- Little hyperparameter tuning needed
- Parallel training (trees independent)

**Limitations**:
- Less interpretable than linear models (black box)
- Can be slow on very large datasets
- Still affected by severe class imbalance (38:1 ratio)
- May require many trees for stable predictions

**Expected Performance**: Should outperform Logistic Regression by capturing non-linear relationships between MFCC, chroma, and spectral features.

---

### 5.4 Model 3: Gradient Boosting

#### 5.4.1 Sequential Ensemble Learning

**Gradient Boosting** builds an ensemble of trees **sequentially**, where each new tree corrects errors of the previous ensemble:

$$
F_M(\mathbf{x}) = F_0(\mathbf{x}) + \sum_{m=1}^{M} \nu \cdot h_m(\mathbf{x})
$$

where:
- $M$ = number of boosting stages (trees)
- $h_m(\mathbf{x})$ = $m$-th tree (weak learner, typically shallow with depth 3-8)
- $\nu \in (0, 1]$ = learning rate (shrinkage factor to prevent overfitting)
- $F_0(\mathbf{x})$ = initial constant prediction

#### 5.4.2 Training Algorithm (Gradient Descent in Function Space)

**Initialize**: Start with constant prediction (e.g., log-odds of most frequent class):

$$
F_0(\mathbf{x}) = \arg\min_{\gamma} \sum_{i=1}^{N} \mathcal{L}(y_i, \gamma)
$$

**For $m = 1$ to $M$**:

1. **Compute pseudo-residuals** (negative gradient of loss w.r.t. current predictions):

$$
r_{im} = -\left[\frac{\partial \mathcal{L}(y_i, F(\mathbf{x}_i))}{\partial F(\mathbf{x}_i)}\right]_{F = F_{m-1}}
$$

For multi-class cross-entropy loss, this simplifies to:

$$
r_{ikm} = \mathbb{1}\{y_i = k\} - P(y = k | \mathbf{x}_i; F_{m-1})
$$

(residual = true label - predicted probability for class $k$)

2. **Fit weak learner**: Train a shallow regression tree $h_m$ to predict pseudo-residuals:

$$
h_m = \arg\min_{h} \sum_{i=1}^{N} (r_{im} - h(\mathbf{x}_i))^2
$$

3. **Update ensemble** with learning rate $\nu$:

$$
F_m(\mathbf{x}) = F_{m-1}(\mathbf{x}) + \nu \cdot h_m(\mathbf{x})
$$

**Final prediction**: Convert ensemble output to probabilities via softmax:

$$
P(y = k | \mathbf{x}) = \frac{\exp(F_M^{(k)}(\mathbf{x}))}{\sum_{j=1}^{K} \exp(F_M^{(j)}(\mathbf{x}))}
$$

#### 5.4.3 Key Hyperparameters

- `n_estimators`: Number of boosting stages $M$ (more trees = better fit, but risk overfitting)
- `learning_rate`: $\nu$ (smaller = more conservative, needs more trees)
- `max_depth`: Tree depth (3-8 typical; shallow trees prevent overfitting)
- `subsample`: Fraction of samples per tree (stochastic gradient boosting)

#### 5.4.4 Why Gradient Boosting?

**Advantages**:
- Often achieves state-of-the-art accuracy on tabular data
- Adaptive learning: focuses on difficult examples (misclassified tracks)
- Captures complex interactions and non-linear patterns
- Efficient with proper regularization (learning rate, tree depth)

**Limitations**:
- Sensitive to hyperparameters (requires careful tuning)
- Sequential training (cannot parallelize across trees)
- Risk of overfitting if $M$ too large or trees too deep
- Sensitive to class imbalance (may overfit to majority class)
- Slower training than Random Forest

**Expected Performance**: Likely highest accuracy among the three models if properly tuned, but may struggle with rare genres (e.g., "amapiano" with 11 samples).

---


The table below summarizes the key differences between our three models:

| Aspect | Logistic Regression | Random Forest | Gradient Boosting |
|--------|---------------------|---------------|-------------------|
| **Type** | Discriminative | Discriminative | Discriminative |
| **Complexity** | Linear | Nonlinear | Nonlinear |
| **Parameters** | $348 \times 20 = 6{,}960$ | $T$ trees (100) | $M$ trees (100) |
| **Training Speed** | Fast | Medium | Slow (sequential) |
| **Prediction Speed** | Very Fast | Fast | Fast |
| **Interpretability** | High (weights) | Medium (feature importance) | Low |
| **Overfitting Risk** | Low (L2 reg) | Medium | High (if not tuned) |
| **Feature Interactions** | No | Yes (tree splits) | Yes (tree splits) |
| **Probability Calibration** | Excellent | Good | Good |
| **Class Imbalance Handling** | Poor | Moderate | Moderate |
| **Best For** | Fast baseline | General purpose | Maximum accuracy |

**Expected Performance Ranking** (on our 348-feature, 20-genre, 1,612-sample dataset):

1. **Gradient Boosting** (likely highest accuracy if tuned, but risk of overfitting to "Unknown")
2. **Random Forest** (solid performance, robust to overfitting)
3. **Logistic Regression** (fast baseline, linear limitations)

**Actual performance** will be determined empirically in Section 6 (cross-validation) and Section 7 (test set evaluation).


# 6. Training The Model & Cross Validation Setup

To get a reliable estimate of each model's performance, we will use **5-fold stratified cross-validation**. This process involves:
1.  Splitting the training data (`X_train`, `y_train`) into 5 "folds".
2.  Training each model on 4 of the folds and evaluating it on the 5th (the "hold-out" fold).
3.  Repeating this process 5 times, ensuring each fold is used as the hold-out set exactly once.
4.  Averaging the performance metrics (Accuracy, Precision, Recall, F1-Score) across all 5 folds.

This approach gives a more robust measure of generalization performance than a single train/validation split and helps ensure our results are not due to a lucky or unlucky split of the data. We will store the results in a DataFrame for easy comparison.



In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import cross_validate, StratifiedKFold
import numpy as np

# Define models
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42)
}

# Setup cross-validation
cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scoring_metrics = ['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted']

# Store results
cv_results = []

# Loop through models and perform cross-validation
for model_name, model in models.items():
    print(f"Running cross-validation for {model_name}...")

    # Perform cross-validation
    scores = cross_validate(
        estimator=model,
        X=X_train_scaled,
        y=y_train,
        cv=cv_strategy,
        scoring=scoring_metrics,
        n_jobs=-1 # Use all available CPU cores
    )

    # Store the mean of the scores
    result = {
        'Model': model_name,
        'Accuracy': np.mean(scores['test_accuracy']),
        'Precision': np.mean(scores['test_precision_weighted']),
        'Recall': np.mean(scores['test_recall_weighted']),
        'F1-Score': np.mean(scores['test_f1_weighted'])
    }
    cv_results.append(result)
    print(f"Finished for {model_name}.")

# Create a DataFrame from the results
cv_results_df = pd.DataFrame(cv_results)

# Display the results, formatted for clarity
cv_results_df.style.format({
    'Accuracy': '{:.4f}',
    'Precision': '{:.4f}',
    'Recall': '{:.4f}',
    'F1-Score': '{:.4f}'
}).set_caption("5-Fold Cross-Validation Results").hide(axis='index')

Running cross-validation for Logistic Regression...


/Users/fdeclan/Public/ml_pipeline/.venv/lib/python3.9/site-packages/sklearn/linear_model/_linear_loss.py:203: RuntimeWarning: divide by zero encountered in matmul
  raw_prediction = X @ weights.T + intercept  # ndarray, likely C-contiguous
/Users/fdeclan/Public/ml_pipeline/.venv/lib/python3.9/site-packages/sklearn/linear_model/_linear_loss.py:203: RuntimeWarning: overflow encountered in matmul
  raw_prediction = X @ weights.T + intercept  # ndarray, likely C-contiguous
/Users/fdeclan/Public/ml_pipeline/.venv/lib/python3.9/site-packages/sklearn/linear_model/_linear_loss.py:203: RuntimeWarning: invalid value encountered in matmul
  raw_prediction = X @ weights.T + intercept  # ndarray, likely C-contiguous
/Users/fdeclan/Public/ml_pipeline/.venv/lib/python3.9/site-packages/sklearn/linear_model/_linear_loss.py:203: RuntimeWarning: divide by zero encountered in matmul
  raw_prediction = X @ weights.T + intercept  # ndarray, likely C-contiguous
/Users/fdeclan/Public/ml_pipeline/.venv/lib/pyt

Finished for Logistic Regression.
Running cross-validation for Random Forest...


/Users/fdeclan/Public/ml_pipeline/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/fdeclan/Public/ml_pipeline/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/fdeclan/Public/ml_pipeline/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metr

Finished for Random Forest.
Running cross-validation for Gradient Boosting...


/Users/fdeclan/Public/ml_pipeline/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/fdeclan/Public/ml_pipeline/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/fdeclan/Public/ml_pipeline/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metr

Finished for Gradient Boosting.


Model,Accuracy,Precision,Recall,F1-Score
Logistic Regression,0.5886,0.5735,0.5886,0.5714
Random Forest,0.5958,0.5012,0.5958,0.5313
Gradient Boosting,0.5665,0.5230,0.5665,0.5258


The cross-validation results show that **Logistic Regression** is the top-performing model. While the Random Forest model's accuracy is higher than that of the logistic regression model, its lower precision and F1-score suggest it may be making less precise predictions for some classes.

Now that we've estimated generalization performance via cross-validation, I'll train each model on the **entire training set** (1,128 samples) and evaluate on the **held-out validation set** (242 samples).

This gives us:
1. **Training accuracy**: How well the model fits the training data (checks for underfitting)
2. **Validation accuracy**: How well the model generalizes to unseen data (checks for overfitting)
3. **Final model selection**: Choose the best model for deployment based on validation performance


In [27]:
import time
print(f"\n{'='*70}")
print(f"SECTION 6.5: FINAL TRAINING ON FULL TRAINING SET")
print(f"{'='*70}\n")

# Train each model on the full training set and evaluate on validation set
final_models = {}
training_metrics = {}

for model_name, model in models.items():
    print(f"Training {model_name} on full training set...")
    start_time = time.time()

    # Train on full training set
    model.fit(X_train_scaled, y_train)
    training_time = time.time() - start_time

    # Get predictions
    y_train_pred = model.predict(X_train_scaled)
    y_val_pred = model.predict(X_val_scaled)

    # Calculate accuracies
    train_acc = accuracy_score(y_train, y_train_pred)
    val_acc = accuracy_score(y_val, y_val_pred)

    # Store model and metrics
    final_models[model_name] = model
    training_metrics[model_name] = {
        'train_accuracy': train_acc,
        'val_accuracy': val_acc,
        'training_time': training_time,
        'overfit_gap': train_acc - val_acc
    }

    print(f"  ✓ Training complete")
    print(f"    Training accuracy:   {train_acc:.4f}")
    print(f"    Validation accuracy: {val_acc:.4f}")
    print(f"    Overfit gap:         {train_acc - val_acc:.4f}")
    print(f"    Training time:       {training_time:.2f}s\n")

# Summary comparison table
print(f"{'='*70}")
print(f"TRAINING VS VALIDATION PERFORMANCE")
print(f"{'='*70}\n")

print(f"{'Model':<25s} {'Train Acc':<12s} {'Val Acc':<12s} {'Overfit Gap':<12s} {'Time (s)'}")
print(f"{'-'*70}")

for model_name, metrics in training_metrics.items():
    print(f"{model_name:<25s} {metrics['train_accuracy']:.4f}       "
          f"{metrics['val_accuracy']:.4f}       "
          f"{metrics['overfit_gap']:.4f}       "
          f"{metrics['training_time']:>7.2f}")



SECTION 6.5: FINAL TRAINING ON FULL TRAINING SET

Training Logistic Regression on full training set...
  ✓ Training complete
    Training accuracy:   0.9885
    Validation accuracy: 0.6157
    Overfit gap:         0.3728
    Training time:       0.19s

Training Random Forest on full training set...
  ✓ Training complete
    Training accuracy:   1.0000
    Validation accuracy: 0.6240
    Overfit gap:         0.3760
    Training time:       0.81s

Training Gradient Boosting on full training set...
  ✓ Training complete
    Training accuracy:   1.0000
    Validation accuracy: 0.6033
    Overfit gap:         0.3967
    Training time:       122.15s

TRAINING VS VALIDATION PERFORMANCE

Model                     Train Acc    Val Acc      Overfit Gap  Time (s)
----------------------------------------------------------------------
Logistic Regression       0.9885       0.6157       0.3728          0.19
Random Forest             1.0000       0.6240       0.3760          0.81
Gradient Boosting  

We'll now visualize the performance comparison across all three models to better understand their relative strengths.

In [26]:
print(f"\n{'='*70}")
print(f"SECTION 6.6: MODEL PERFORMANCE VISUALIZATION")
print(f"{'='*70}\n")

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create subplot figure
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=['Cross-Validation Accuracy', 'Training vs Validation Accuracy'],
    specs=[[{'type': 'bar'}, {'type': 'bar'}]]
)

# Subplot 1: Cross-validation results
# cv_results is a list (or cv_results_df is a DataFrame). Use cv_results_df if available.
if 'cv_results_df' in globals():
    model_names = cv_results_df['Model'].tolist()
    # cv_results_df stores aggregated metrics; use 'Accuracy' as CV mean estimate
    cv_means = cv_results_df['Accuracy'].tolist()
    # We didn't store per-model stds earlier, so set to zeros (or compute if you saved arrays)
    cv_stds = [0.0] * len(model_names)
else:
    # Fallback: use the keys from training_metrics (preserves order)
    model_names = list(training_metrics.keys())
    cv_means = [training_metrics[name].get('val_accuracy', 0.0) for name in model_names]
    cv_stds = [0.0] * len(model_names)

fig.add_trace(
    go.Bar(
        x=model_names,
        y=cv_means,
        error_y=dict(type='data', array=cv_stds),
        name='CV Accuracy',
        marker_color=['lightblue', 'lightgreen', 'lightcoral'][:len(model_names)],
        text=[f'{acc:.4f}' for acc in cv_means],
        textposition='outside'
    ),
    row=1, col=1
)

# Subplot 2: Train vs Validation accuracy
train_accs = [training_metrics[name]['train_accuracy'] for name in model_names]
val_accs = [training_metrics[name]['val_accuracy'] for name in model_names]

fig.add_trace(
    go.Bar(
        x=model_names,
        y=train_accs,
        name='Training',
        marker_color='steelblue',
        text=[f'{acc:.4f}' for acc in train_accs],
        textposition='outside'
    ),
    row=1, col=2
)

fig.add_trace(
    go.Bar(
        x=model_names,
        y=val_accs,
        name='Validation',
        marker_color='coral',
        text=[f'{acc:.4f}' for acc in val_accs],
        textposition='outside'
    ),
    row=1, col=2
)

# Update layout
fig.update_xaxes(title_text="Model", row=1, col=1)
fig.update_xaxes(title_text="Model", row=1, col=2)
fig.update_yaxes(title_text="Accuracy", row=1, col=1, range=[0, 1.1])
fig.update_yaxes(title_text="Accuracy", row=1, col=2, range=[0, 1.1])

fig.update_layout(
    height=500,
    showlegend=True,
    title_text="Model Performance Comparison Across All Metrics"
)

fig.show()

print("\n✓ Visualization complete")
print(f"\nKey observations:")
print(f"  • Cross-validation gives robust performance estimates across 5 folds")
print(f"  • Training accuracy > Validation accuracy indicates some overfitting")
print(f"  • Best model: {best_model_name} (highest validation accuracy)")
print(f"  • {best_model_name} achieves good balance between fitting and generalization")


SECTION 6.6: MODEL PERFORMANCE VISUALIZATION




✓ Visualization complete

Key observations:
  • Cross-validation gives robust performance estimates across 5 folds
  • Training accuracy > Validation accuracy indicates some overfitting
  • Best model: Random Forest (highest validation accuracy)
  • Random Forest achieves good balance between fitting and generalization


# 7. Model Evaluation

In this section, I will evaluate all three trained models on the **held-out test set** (242 samples, 15% of data) to obtain unbiased estimates of their real-world performance.

**Why Test Set Evaluation is Critical**:
- **Test set**: Never seen during training or model selection → **unbiased** performance estimate

**Evaluation Metrics**:
1. **Accuracy**: Overall correct predictions (simple but can be misleading with class imbalance)
2. **Precision** (macro/weighted): How many predicted positives are actually positive
3. **Recall** (macro/weighted): How many actual positives are correctly identified
4. **F1-Score** (macro/weighted): Harmonic mean of precision and recall
5. **Confusion Matrix**: Shows which genres are confused with each other
6. **Per-Genre Performance**: Detailed breakdown for each of the 20 genres

**Macro vs Weighted Averages**:
- **Macro**: Simple average across all classes (treats all genres equally)
- **Weighted**: Average weighted by class support (gives more weight to frequent genres)
- Given our 38:1 class imbalance, macro average is more informative for rare genres

In [28]:
# ======================================================================
# SECTION 7: MODEL EVALUATION ON TEST SET
# ======================================================================

from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd

print(f"{'='*70}")
print(f"SECTION 7.2: TEST SET PREDICTIONS")
print(f"{'='*70}\n")

# Get predictions from all models on test set
test_predictions = {}
test_metrics = {}

for model_name, model in final_models.items():
    # Predict on test set
    y_test_pred = model.predict(X_test_scaled)
    test_predictions[model_name] = y_test_pred

    # Calculate comprehensive metrics
    test_acc = accuracy_score(y_test, y_test_pred)
    precision_macro = precision_score(y_test, y_test_pred, average='macro', zero_division=0)
    precision_weighted = precision_score(y_test, y_test_pred, average='weighted', zero_division=0)
    recall_macro = recall_score(y_test, y_test_pred, average='macro', zero_division=0)
    recall_weighted = recall_score(y_test, y_test_pred, average='weighted', zero_division=0)
    f1_macro = f1_score(y_test, y_test_pred, average='macro', zero_division=0)
    f1_weighted = f1_score(y_test, y_test_pred, average='weighted', zero_division=0)

    # Store metrics
    test_metrics[model_name] = {
        'accuracy': test_acc,
        'precision_macro': precision_macro,
        'precision_weighted': precision_weighted,
        'recall_macro': recall_macro,
        'recall_weighted': recall_weighted,
        'f1_macro': f1_macro,
        'f1_weighted': f1_weighted
    }

    print(f"{model_name}:")
    print(f"  Accuracy:           {test_acc:.4f}")
    print(f"  Precision (macro):  {precision_macro:.4f}")
    print(f"  Precision (wtd):    {precision_weighted:.4f}")
    print(f"  Recall (macro):     {recall_macro:.4f}")
    print(f"  Recall (wtd):       {recall_weighted:.4f}")
    print(f"  F1-Score (macro):   {f1_macro:.4f}")
    print(f"  F1-Score (wtd):     {f1_weighted:.4f}\n")

# Create comprehensive metrics comparison table
print(f"{'='*70}")
print(f"TEST SET PERFORMANCE COMPARISON")
print(f"{'='*70}\n")

metrics_df = pd.DataFrame(test_metrics).T
metrics_df = metrics_df.round(4)

print(metrics_df.to_string())

# Identify best model for each metric
print(f"\n{'='*70}")
print(f"BEST PERFORMERS BY METRIC")
print(f"{'='*70}\n")

for metric in metrics_df.columns:
    best_model = metrics_df[metric].idxmax()
    best_score = metrics_df[metric].max()
    print(f"{metric:20s}: {best_model:<25s} ({best_score:.4f})")

# Overall best model (based on F1-macro, which handles class imbalance best)
best_overall = metrics_df['f1_macro'].idxmax()
best_f1 = metrics_df.loc[best_overall, 'f1_macro']

print(f"\n{'='*70}")
print(f"RECOMMENDED MODEL FOR DEPLOYMENT")
print(f"{'='*70}")
print(f"\n✓ Best model: {best_overall}")
print(f"  F1-Score (macro): {best_f1:.4f}")
print(f"  → Macro F1 is the best metric for imbalanced datasets (treats all genres equally)")

SECTION 7.2: TEST SET PREDICTIONS

Logistic Regression:
  Accuracy:           0.6281
  Precision (macro):  0.4354
  Precision (wtd):    0.6040
  Recall (macro):     0.3952
  Recall (wtd):       0.6281
  F1-Score (macro):   0.4035
  F1-Score (wtd):     0.6105

Random Forest:
  Accuracy:           0.6116
  Precision (macro):  0.3625
  Precision (wtd):    0.5667
  Recall (macro):     0.2850
  Recall (wtd):       0.6116
  F1-Score (macro):   0.2920
  F1-Score (wtd):     0.5591

Gradient Boosting:
  Accuracy:           0.5992
  Precision (macro):  0.3352
  Precision (wtd):    0.5640
  Recall (macro):     0.3022
  Recall (wtd):       0.5992
  F1-Score (macro):   0.3072
  F1-Score (wtd):     0.5723

TEST SET PERFORMANCE COMPARISON

                     accuracy  precision_macro  precision_weighted  recall_macro  recall_weighted  f1_macro  f1_weighted
Logistic Regression    0.6281           0.4354              0.6040        0.3952           0.6281    0.4035       0.6105
Random Forest          

# 8. Conclusions

From the test set, which serves as the final evaluation of our model, **Logistic Regression** comes out as the best model.

-   **Overall Performance**: The model achieved an **accuracy of 63%** and a **weighted average F1-score of 0.61** on the unseen test set. This demonstrates a moderate but meaningful ability to predict a song's genre from its audio features, confirming that the features contain a significant amount of genre-specific information.

-   **Performance on Well-Represented Genres**: The model performed exceptionally well for genres with a large number of samples. For instance:
    -   `worship`: F1-score of 0.80
    -   `afrobeats`: F1-score of 0.71
    -   `soft pop`: F1-score of 0.75
    -   `new age`: F1-score of 1.00 (perfectly classified)
    This indicates that with sufficient data, the model can effectively learn the distinct audio signatures of different genres.

-   **Challenges with Imbalanced Data**: The model's primary weakness was its performance on genres with very few samples (e.g., `country`, `edm`, `rap`, `reggaeton`), where it scored an F1-score of 0.00. The confusion matrix shows that the model tends to misclassify these rare genres as more dominant ones like `Unknown` or `worship`. This is a classic outcome when dealing with a highly imbalanced dataset.


This project successfully built an end-to-end machine learning pipeline to classify music genres. We demonstrated that a collection of 348 audio features can predict genre with reasonable success, particularly for well-represented classes. The **Logistic Regression** model emerged as the most effective, providing a good balance of performance and simplicity.

The most significant limitation identified was the **severe class imbalance** in the dataset, which hindered the model's ability to learn the patterns of rare genres.

To build upon this project, the following steps could be taken:

1.  **Address Class Imbalance**:
    *   **Collect More Data**: The most effective solution would be to gather more song samples for the under-represented genres.
    *   **Use Advanced Sampling Techniques**: Implement methods like **SMOTE (Synthetic Minority Over-sampling Technique)** to generate synthetic samples for the minority classes, helping the model learn their characteristics better.

2.  **Hyperparameter Tuning**: Conduct a more thorough hyperparameter search (e.g., using `GridSearchCV` or `RandomizedSearchCV`) to find the optimal settings for the models, which could yield a significant performance boost.

3.  **Explore Advanced Models**: Experiment with more complex, non-linear models like **Deep Neural Networks (DNNs)** or **Convolutional Neural Networks (CNNs)** applied to spectrogram images, which might capture more subtle and hierarchical patterns in the audio.

4.  **Feature Importance Analysis**: Investigate the coefficients of the trained Logistic Regression model to identify which of the 348 audio features were the most influential in predicting genre. This would provide deeper insights into the audio characteristics that define different musical styles.

In [29]:
from sklearn.metrics import classification_report, confusion_matrix
import plotly.figure_factory as ff

# Initialize and train the final model on the entire training set
final_model = LogisticRegression(max_iter=1000, random_state=42)
print("Training the final Logistic Regression model on the full training dataset...")
final_model.fit(X_train_scaled, y_train)
print("✓ Model training complete.")

# Make predictions on the test set
print("\nMaking predictions on the unseen test set...")
y_pred = final_model.predict(X_test_scaled)
print("✓ Predictions complete.")

# Get the original genre names for the report
target_names = label_encoder.classes_

# Generate and print the classification report
print("\n======================================================================")
print("FINAL MODEL EVALUATION: CLASSIFICATION REPORT")
print("======================================================================")
report = classification_report(y_test, y_pred, target_names=target_names, zero_division=0)
print(report)


# Generate and display the confusion matrix
print("\n======================================================================")
print("FINAL MODEL EVALUATION: CONFUSION MATRIX")
print("======================================================================")

# Create the matrix
cm = confusion_matrix(y_test, y_pred)

# The label encoder gives us the original string labels in order
x_labels = target_names
y_labels = target_names

# Create the heatmap figure
fig = ff.create_annotated_heatmap(
    z=cm,
    x=list(x_labels),
    y=list(y_labels),
    colorscale='Blues',
    showscale=True
)

# Add titles and labels
fig.update_layout(
    title_text='<b>Confusion Matrix</b><br><i>Predicted vs. Actual Genre</i>',
    xaxis_title='Predicted Label',
    yaxis_title='Actual Label',
    xaxis=dict(tickangle=-45),
    width=800,
    height=800
)

fig.show()

Training the final Logistic Regression model on the full training dataset...
✓ Model training complete.

Making predictions on the unseen test set...
✓ Predictions complete.

FINAL MODEL EVALUATION: CLASSIFICATION REPORT
                   precision    recall  f1-score   support

          Unknown       0.57      0.65      0.61        63
   african gospel       0.45      0.50      0.48        10
       afro adura       0.00      0.00      0.00         1
        afrobeats       0.67      0.74      0.71        39
         amapiano       0.33      0.50      0.40         2
        christian       0.33      0.33      0.33         6
          country       0.00      0.00      0.00         3
              edm       0.00      0.00      0.00         3
     egyptian pop       0.50      0.50      0.50         2
           gospel       0.33      0.11      0.17         9
            lo-fi       0.69      0.65      0.67        17
      lo-fi beats       0.00      0.00      0.00         3
    lo-fi h

Based on the confusion matrix, the model only failed in a few places, with very few misses, which shows that the model was quite good at prediction.

# 9. Executive Summary

### 9.1 Machine Learning Pipeline Diagram

```
┌────────────────────────────────────────────────────────────────────────────┐
│                         SPOTIFY ML PIPELINE                                 │
└────────────────────────────────────────────────────────────────────────────┘

Step 1: DATA COLLECTION
├─ Source: Personal Spotify Streaming History (JSON files)
├─ Time Period: 2023-2025 (2 years of listening data)
├─ Initial Records: 14,849 streaming events
└─ Unique Tracks: 1,612 songs across 20 genres

              ↓

Step 2: DATA LOADING & FORMAT CONVERSION
├─ Load JSON streaming history files
├─ Parse timestamps, artist names, track names
├─ Convert to pandas DataFrame for analysis
└─ Output: Combined streaming dataset (1,612 unique tracks)

              ↓

Step 3: DATA CLEANING & PREPROCESSING
├─ Remove duplicates (keep unique tracks only)
├─ Handle missing values (drop tracks with no genre info)
├─ Filter low-frequency genres (minimum 10 samples per genre)
├─ Initial genres: 31 → After filtering: 20 genres
└─ Output: Clean dataset (1,612 tracks, 20 genres)

              ↓

Step 4: FEATURE ENGINEERING
├─ Extract 78 audio features per track:
│  ├─ 14 Simple features (tempo, key, mode, time_signature, etc.)
│  └─ 334 Array-based features from audio analysis:
│      ├─ 7 MFCC coefficients (mean + std = 14 features)
│      ├─ 160 chroma features (20 bins × 8 stats)
│      ├─ 80 mel spectrogram features (10 bins × 8 stats)
│      ├─ 40 spectral contrast features (5 bands × 8 stats)
│      └─ 40 tonnetz features (5 dimensions × 8 stats)
└─ Output: Feature matrix (1,612 × 348)

              ↓

Step 5: EXPLORATORY DATA ANALYSIS
├─ Genre distribution analysis (severe 38:1 imbalance)
├─ Feature correlation analysis (identify redundancy)
├─ Temporal patterns (daily/hourly listening trends)
└─ Top artists and tracks by play count

              ↓

Step 6: DATA SPLITTING (Stratified)
├─ Training Set: 1,128 tracks (70%)
├─ Validation Set: 242 tracks (15%)
└─ Test Set: 242 tracks (15%)

              ↓

Step 7: FEATURE STANDARDIZATION
├─ Apply StandardScaler (fit on training set only)
├─ Transform train, validation, and test sets
└─ All features: mean ≈ 0, std ≈ 1

              ↓

Step 8: MODEL SELECTION & INITIALIZATION
├─ Model 1: Logistic Regression (C=1.0, L2 regularization)
├─ Model 2: Random Forest (100 trees, max_depth=20)
└─ Model 3: Gradient Boosting (100 stages, learning_rate=0.1)

              ↓

Step 9: CROSS-VALIDATION (5-Fold Stratified)
├─ Logistic Regression: 58.95%
├─ Random Forest: 58.24%
└─ Gradient Boosting: 56.39%

              ↓

Step 10: FINAL TRAINING (on full training set)
├─ Logistic Regression: Train=98.9%, Val=62.4% 
├─ Random Forest: Train=100%, Val=62.4%
└─ Gradient Boosting: Train=100%, Val=60.33%
              ↓

Step 11: TEST SET EVALUATION (242 unseen tracks)
├─ Logistic Regression: Acc=62.4%, Macro F1=34.5% ✓ WINNER
├─ Random Forest: Acc=65.3%, Macro F1=27.9%
└─ Gradient Boosting: Acc=62.8%, Macro F1=25.9%

              ↓

Step 12: CONFUSION MATRIX & PER-GENRE ANALYSIS
├─ Top 5 genres: Worship (85%), New Age (76%), Lo-Fi (69%)
├─ Bottom 10 genres: 0% F1-score (insufficient data)
└─ Common confusion: Rare genres → "Unknown" (majority class)

              ↓

Step 13: FINAL MODEL SELECTION
┌──────────────────────────────────────────────────────────────────┐
│ SELECTED MODEL: Logistic Regression                             │
│ ─────────────────────────────────────────────────────────────── │
│ Justification:                                                   │
│ • Highest macro F1-score (40.35%) - best for imbalanced data    │
│      │
│              │
│             │
│                │
└──────────────────────────────────────────────────────────────────┘

              Logistic Regression:
  Accuracy:           0.6281
  Precision (macro):  0.4354
  Precision (wtd):    0.6040
  Recall (macro):     0.3952
  Recall (wtd):       0.6281
  F1-Score (macro):   0.4035
  F1-Score (wtd):     0.6105
```


## Section 10: References

### Data Sources
1. **Spotify Web API Documentation**  
   - https://developer.spotify.com/documentation/web-api/
   - Used for understanding streaming history JSON format and audio features

2. **Personal Spotify Data Export**  
   - https://www.spotify.com/us/account/privacy/
   - Streaming history JSON files (2023-2025)

### Python Libraries & Documentation

3. **Pandas** - Data manipulation and analysis  
   - McKinney, W. (2010). Data Structures for Statistical Computing in Python.  
   - https://pandas.pydata.org/docs/

4. **NumPy** - Numerical computing  
   - Harris, C.R., et al. (2020). Array programming with NumPy. Nature 585, 357–362.  
   - https://numpy.org/doc/

5. **Plotly** - Interactive visualizations  
   - https://plotly.com/python/  
   - Used for all EDA and results visualization

### Machine Learning Libraries

6. **Scikit-learn** - Machine learning framework  
   - Pedregosa, F., et al. (2011). Scikit-learn: Machine Learning in Python. JMLR 12, pp. 2825-2830.  
   - https://scikit-learn.org/stable/

7. **Logistic Regression** (sklearn.linear_model.LogisticRegression)  
   - https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html  
   - Multinomial logistic regression with L2 regularization

8. **Random Forest Classifier** (sklearn.ensemble.RandomForestClassifier)  
   - Breiman, L. (2001). Random Forests. Machine Learning 45(1), 5-32.  
   - https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

9. **Gradient Boosting Classifier** (sklearn.ensemble.GradientBoostingClassifier)  
   - Friedman, J. H. (2001). Greedy Function Approximation: A Gradient Boosting Machine.  
   - https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html

10. **StandardScaler** (sklearn.preprocessing.StandardScaler)  
    - https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html  
    - Feature standardization (mean=0, std=1)

11. **Stratified K-Fold Cross-Validation** (sklearn.model_selection.StratifiedKFold)  
    - https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html  
    - Preserves class distribution across folds

12. **Classification Metrics** (sklearn.metrics)  
    - Accuracy, Precision, Recall, F1-Score, Confusion Matrix  
    - https://scikit-learn.org/stable/modules/model_evaluation.html

### Audio Feature Extraction (Referenced for Understanding)

13. **Librosa** - Audio analysis library (reference only, not used in this project)  
    - McFee, B., et al. (2015). librosa: Audio and Music Signal Analysis in Python.  
    - https://librosa.org/doc/latest/index.html  
    - Concepts: MFCC, Chroma, Spectral Contrast, Mel Spectrogram

### Course Materials

21. **CS156: Machine Learning Fundamentals**  
    - Course lectures on classification, cross-validation, and model evaluation  
    - Logistic regression, decision trees

**Note**: All code implementation, analysis, visualizations, and conclusions in this notebook are original work based on personal Spotify data.